In [ ]:
from datetime import datetime
from pathlib import Path
import pprint
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
from pyseir.models.seir_model import SEIRModel
from pyseir.parameters.parameter_ensemble_generator import ParameterEnsembleGenerator
from pyseir.models import suppression_policies

In [ ]:
from pyseir.outcomes import OutcomesSampler, ContinuousParameter, OutcomeModels

In [ ]:
t_list = np.linspace(0, 1080, 1080)
parameter_generator = ParameterEnsembleGenerator(fips='06037',
                                                 N_samples=1000,
                                                 t_list=t_list)

parameter_space = [ContinuousParameter('R0', 2, 4), 
                   ContinuousParameter('delta', 1.0 / 30, 1), 
                   ContinuousParameter('suppression_policy', 0.01, 1)]

def total_deaths(rollout: np.array) -> float:
    return np.log10(rollout[-1])

def max_hicu(rollout: np.array) -> float:
    return np.log10(np.max(rollout))

def time_to_max_hicu(rollout: np.array) -> int:
    return np.log10(np.argmax(rollout))

outcome_fs = {"D": total_deaths, 
              "HICU-1": max_hicu, 
              "HICU-2": time_to_max_hicu}

In [ ]:
%%time
os = OutcomesSampler(parameter_generator, parameter_space, outcome_fs, num_samples=1000)

In [ ]:
model_base = Path.cwd() / Path("models/sensitivity_analysis")
ts_path = model_base / Path("{:%Y%m%d-%H%M}".format(datetime.now()))
ts_path.mkdir(parents=True, exist_ok=True)

In [ ]:
samples_path = Path("outcome_samples.pkl")
joblib.dump(os, ts_path / samples_path, compress=True)

In [ ]:
%%time

def fn_approximator(oc, X, y):
    kernel = RationalQuadratic(length_scale=0.75)
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=10)
    gpr.fit(X, y)
    return (oc, gpr)

om = OutcomeModels(os.outcomes_df, parameter_space, fn_approximator)

In [ ]:
models_path = Path("outcome_models.pkl")
joblib.dump(om.outcome_models, ts_path / models_path, compress=9)

In [ ]:
sp_sensitivity = pd.DataFrame({"R0": 2.5 * np.ones(500), "delta": 1/14 * np.ones(500), "suppression_policy": np.linspace(0.01, 1, 500)})

In [ ]:
x_ = sp_sensitivity["suppression_policy"]
y_mean, y_cov = om.outcome_models["max_hicu"].predict(sp_sensitivity, return_cov=True)
_ = plt.plot(x_, y_mean)
_ = plt.fill_between(x_, y_mean - np.sqrt(np.diag(y_cov)),
                     y_mean + np.sqrt(np.diag(y_cov)),
                     alpha=0.5, color='k')
_ = plt.xlabel("Suppression Policy")
_ = plt.ylabel("log max(HICU)")

In [ ]:
x_ = sp_sensitivity["suppression_policy"]
y_mean, y_cov = om.outcome_models["total_deaths"].predict(sp_sensitivity, return_cov=True)
_ = plt.plot(x_, y_mean)
_ = plt.fill_between(x_, y_mean - np.sqrt(np.diag(y_cov)),
                     y_mean + np.sqrt(np.diag(y_cov)),
                     alpha=0.5, color='k')
_ = plt.xlabel("Suppression Policy")
_ = plt.ylabel("log total deaths")

In [ ]:
x_ = sp_sensitivity["suppression_policy"]
y_mean, y_cov = om.outcome_models["time_to_max_hicu"].predict(sp_sensitivity, return_cov=True)
_ = plt.plot(x_, y_mean)
_ = plt.fill_between(x_, y_mean - np.sqrt(np.diag(y_cov)),
                     y_mean + np.sqrt(np.diag(y_cov)),
                     alpha=0.5, color='k')
_ = plt.xlabel("Suppression Policy")
_ = plt.ylabel("time to max HICU")
